In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# Load the data
genotypes = pd.read_csv('Genotypic_data_maf10_min10_291acc.txt', index_col=0)
phenotype = pd.read_csv('phenodata_BLUP_2012.txt', sep='\t', index_col='ID')


In [3]:
# Define functions for MAF and LD pruning
def calculate_maf(df):
    maf = df.apply(lambda x: min(x.mean(), 1 - x.mean()), axis=0)
    return maf

def ld_pruning(df, threshold=0.5):
    corr = df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return df.drop(to_drop, axis=1)


In [4]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(genotypes, phenotype['FREAR'], test_size=0.2, random_state=42)


In [5]:
# Apply MAF and LD pruning to training data
maf = calculate_maf(X_train)
X_train_filtered = X_train.loc[:, maf > 0.05]
X_train_pruned = ld_pruning(X_train_filtered)

In [7]:
# Perform GWAS
gwas_results = pd.DataFrame(index=X_train_pruned.columns, columns=['p_value'])
for snp in X_train_pruned.columns:
    x = sm.add_constant(X_train_pruned[snp])
    model = sm.OLS(y_train, x).fit()
    gwas_results.loc[snp, 'p_value'] = model.pvalues[1]

# Apply Benjamini-Hochberg correction
rejected, corrected_p_values = fdrcorrection(gwas_results['p_value'].astype(float), alpha=0.05, method='indep')
gwas_results['corrected_p_value'] = corrected_p_values

In [8]:
# Select SNPs based on corrected p-values
significant_snps = gwas_results.index[rejected]
j = gwas_results.loc[significant_snps]
significant_snp_ids = list(j.index)
pd.Series(significant_snp_ids).to_csv('Features_selected/significant_snp_gwas.csv', index=False)


In [9]:
# Continue with the selected SNPs
X_train_significant = X_train_pruned[significant_snps]
X_test_significant = X_test[significant_snps]

In [16]:
X_train_significant.head()

,IIT9215,IIT12528,IIT10796,solcap_snp_sl_60078,IIT8711,solcap_snp_sl_16331,IIT11440,solcap_snp_sl_31775,solcap_snp_sl_24721,solcap_snp_sl_10533,...,IIT13011,solcap_snp_sl_6371,solcap_snp_sl_19250,solcap_snp_sl_7366,solcap_snp_sl_15381,solcap_snp_sl_15451,IIT9900,IIT9983,IIT9322,solcap_snp_sl_19031
SYNAGAD_214,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
SYNAGAD_223,1.0,1.0,1.0,0.0,0.0,1.0,0.5,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
SYNAGAD_220,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
SYNAGAD_076,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
SYNAGAD_026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Impute missing data and scale the data
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_significant), columns=significant_snps)
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imputed), columns=significant_snps)
X_test_imputed = pd.DataFrame(imputer.transform(X_test_significant), columns=significant_snps)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_imputed), columns=significant_snps)


In [11]:
# Train a Linear Regression model with cross-validation
regressor = LinearRegression()
cv_scores = cross_val_score(regressor, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')


In [12]:
# Fit the model on the entire training set and evaluate on the test set
regressor.fit(X_train_scaled, y_train)
y_pred = regressor.predict(X_test_scaled)


In [13]:
# Calculate RMSE for cross-validation
cv_rmse = np.sqrt(-cv_scores)
print(f"Cross-Validated RMSE: {cv_rmse.mean()}")
# Calculate RMSE and R-squared for the test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
test_r2 = r2_score(y_test, y_pred)

print(f"Test RMSE: {test_rmse}")
print(f"Test R²: {test_r2}")

Cross-Validated RMSE: 2.305880750985638
Test RMSE: 2.2281200965801315
Test R²: 0.2988320346567136
